In [ ]:
#from deva.fileio import load_scenario
# now using a synthetic scenario
import ipywidgets as widgets
from IPython.display import HTML, display
import numpy as np
import pandas as pd
from itertools import permutations, combinations


display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))

# Note to self: jobs only has 4 models

candidates, meta = load_scenario("jobs", None)
#candidates, meta = load_scenario("simple_fraud", None)
metrics = meta['metrics']
attributes = sorted(list(candidates[0].attributes))
candidates

# This function was removed from the codebase - pasted in here

def tabulate_metrics(models):
    # Make a pandas dataframe of the model scores
    ref_model = models[0]
    metrics = list(ref_model.attributes)
    
    model_ids = []
    scores = np.zeros((len(models), len(metrics)))
    # Make array of scores
    for i, m in enumerate(models):
        model_ids.append(m.name)
        for j, metric in enumerate(metrics):
            scores[i, j] = m[metric]

    scores_df = pd.DataFrame(scores, columns=metrics)
    scores_df["model_id"] = np.array(model_ids)
    scores_df.set_index("model_id", inplace=True)

    # Heuristic to detect int columns
    int_metrics = [m for m in metrics if np.max(np.abs(scores_df[m] - scores_df[m].astype(int))) < 1e-4]
    
    # Pandas has per-column data types
    scores_df[int_metrics] = scores_df[int_metrics].astype(int)

    return scores_df



table = tabulate_metrics(candidates)[attributes]
sign = np.array([-1 if metrics[v]["higherIsBetter"] else 1
                 for v in attributes])
scores = table * sign[None, :]

In [ ]:
# Minimum toy problem
attributes = ["X", "Y", "Z"]
sign = [1, 1, 1]
vals = [1,2,3]
scores = list(permutations(vals))
scores.append((2,2,2))  # neutrino
scores = pd.DataFrame(scores)
scores.columns = attributes
table = scores
scores

In [ ]:
# Larger toy problem
attributes = ["W-", "X-", "Y+", "Z-"]
sign = np.array([1, 1, 1, 1], dtype=int)  # Now test with a -1 to make sure "higherisbetter" applies
vals = [1, 2, 3, 4]

scores = np.array(list(permutations(vals)))

# Modify column 3 so scores is "higherisbetter"
sign[2] = -1
scores[:, 2] = 5 - scores[:, 2]

#scores.append((3,3,3,3))  # neutrino
scores = pd.DataFrame(scores)
scores.columns = attributes


table = scores.copy()
# if we flip the higherisbeter column then
# we want to minimise everything
scores *= sign[None, :]
scores

In [ ]:
def make_sliders():
    # Make the sliders
    sliders = []
    col_vals = []  # store a mapping from values to slider labels
    col_labels = []
    
    for i, a in enumerate(attributes):
        col = np.unique(table[a])
        col_ext = np.unique(
            list(np.linspace(min(col), max(col), 20).astype(col.dtype)) +
            list(col)
        )

        if sign[i] < 0:  # "higherisbetter"
            col = col[::-1]
            col_ext = col_ext[::-1]
        
        fmt = '{:d}' if "int" in str(col.dtype) else '{:.2f}'
        opts = [fmt.format(v) for v in col_ext]
        
        # Save for later value-setting
        col_vals.append(col_ext)
        col_labels.append(opts)
        
        # Make the slider
        slider = widgets.SelectionSlider(
            options=opts, value=opts[-1], description=a, readout=True,
            layout=widgets.Layout(width='50%'), continuous_update=True,
            readout_format=fmt,
        )
        sliders.append(slider)
    
    def setval(i, value):
        # Set the slider value
        # requires value to be sign multiplied (as is the use case below)
        # print(f"Setting {attributes[i]} to {value*sign[i]}")
        val_ind = np.argmax(col_vals[i] * sign[i] >= value)
        sliders[i].value = col_labels[i][val_ind]
                
    return sliders, setval

In [ ]:
def make_interactive():
    manual = list(scores.max())  # What the user tried to set
    thresh = manual.copy()  # where the sliders actually are
    priority = list(range(len(attributes)))  # last touched order
        
    # Set up interactive
    sliders, setval = make_sliders()
    lock = [False]  # Mutable lock object for callback function (see below)

    def callback(change):

        if lock[0]:
            return

        lock[0] = True
        
        ix = sliders.index(change['owner'])  # which attribute changed?
        value = float(change['new'])  # what value did it change to?

        # Update the priority order
        priority.remove(ix)
        priority.append(ix)

        # Store the user's requested value
        manual[ix] = value * sign[ix]

        # Keep a working list of which candidates are not rejected
        active = np.ones(scores.shape[0], dtype=bool)

        # Pass over the attribute dimensions in order of recently touched
        # Recent being towards the end of the list
        for p in priority[::-1]:
            col = scores.values[:, p]
            limit = col[active].min()  # best feasible value for this column
            thresh[p] = max(manual[p], limit)  # take the worse of requested and feasible
            active &= (col <= thresh[p])  # filter
            if p != ix:
                setval(p, thresh[p])  # setval likes negative numbers
        lock[0] = False

    for s in sliders:
        s.observe(callback, names='value')


    return widgets.VBox(sliders)

make_interactive()  # worst values on the left

## Manual conflict resolution - identify blocking

This automatic trading off is nice, but it might melt the users brain - try something more manual.


In [ ]:
def make_interactive():
    
    # Start at settings that would include everything
    thresh = scores.values.max(axis=0)
    sliders, setval = make_sliders()
    
    importance = 1. / scores.values.std(axis=0) # heuristic for suggesting fixes
    
    lock = [False]
    
    def callback(change):

        # Allow this callback to change other sliders without recursing
        if lock[0]:
            return
        lock[0] = True
        
        # Interpret the callback data
        ix = sliders.index(change['owner'])  # which attribute changed?
        value = float(change['new']) * sign[ix]  # what value did it change to?
        
        # What is the best we can do on this dimension
        active = (scores.values <= thresh[None, :]).all(axis=1)
        best = scores.values[:,ix][active].min()
        bounced = value < best
            
        if bounced:
            thresh[ix] = best
            setval(ix, best)
        else:
            thresh[ix] = value

        # Determine which thresholds are blocking the last bounce from improving
        targets = scores.values[scores.values[:, ix] < best]
        if targets.shape[0] and bounced:
            targets = np.unique(np.maximum(targets, thresh[None, :]), axis=0)

            # It's actually fairly ill posed - there are MANY possible solutions
            # Lets suggest any tied for least invasive
            delta = ((targets-thresh[None, :])**2).dot(importance)
            least_invasive = delta-delta.min()<1e-5  # in case of ties, suggest multiple
            
            # check if it blocks on any of the "best" candidates
            blocking = thresh < targets[least_invasive].max(axis=0)
        else:
            blocking = np.zeros(len(thresh), dtype=bool)

        # Apply the indicators
        for b, s in zip(blocking, sliders):
            col = 'red' if b else 'white'
            s.style=widgets.SliderStyle(handle_color=col)


        # end of callback - release lock
        lock[0] = False

    # Bind the sliders to the observe function
    for s in sliders:
        s.observe(callback, names='value')

    # Display the sliders
    return widgets.VBox(sliders)

make_interactive()  # worst values are on the left

## More statefule - remembers blocker and blockee

The above can be a bit annoying as in we don't know when the problem has been fixed yet.
It might actually be helpful to identify who is blocked so we know how far to move the levers...

Now Tracking:
- who is currently blocked [orange]
- who was the last bounced [red]
- who is currently blocking the last bounced [black]
- everyone else [white]

In [ ]:
def make_interactive():
    
    # Start at settings that would include everything
    thresh = scores.values.max(axis=0)
    sliders, setval = make_sliders()
    importance = 1. / scores.values.std(axis=0) # heuristic for suggesting fixes
    dims = len(thresh)
    lock = [False]
    lastbounced = [-1]  # negative means no last bounce recorded
    
    def callback(change):

        # Allow this callback to change other sliders without recursing
        if lock[0]:
            return
        lock[0] = True
        
        # Interpret the callback data
        ix = sliders.index(change['owner'])  # which attribute changed?
        value = float(change['new']) * sign[ix]  # what value did it change to?
        
        # What is the best we can do on this dimension
        active = (scores.values <= thresh[None, :]).all(axis=1)
        best = scores.values[:,ix][active].min()
        bounced = value < best
            
        if bounced:
            thresh[ix] = best
            setval(ix, best)
            lastbounced[0] = ix  # remember the bounce
        else:
            thresh[ix] = value
            if lastbounced[0] == ix:
                    lastbounced[0] = -1  # forget the bounce
                    
        # Now what is the best we can do on any dimension?
        active = (scores.values <= thresh[None, :]).all(axis=1)
        optimal = scores.values[active].min(axis=0) == thresh
        
        # Check if another change has resolved the most recent bounce
        if lastbounced[0] >= 0 and not optimal[lastbounced]:
            lastbounced[0] = -1  # forget the bounce
        
        # Determine which thresholds are blocking the last bounce from improving
        targets = scores.values[scores.values[:, lastbounced[0]] < thresh[lastbounced[0]]]
        if targets.shape[0] and lastbounced[0] >= 0:
            targets = np.unique(np.maximum(targets, thresh[None, :]), axis=0)

            # It's actually fairly ill posed - there are MANY possible solutions
            # Lets suggest any tied for least invasive
            delta = ((targets-thresh[None, :])**2).dot(importance)
            least_invasive = delta-delta.min()<1e-5
            
            # check if it blocks on any of the "best" candidates
            blocking = thresh < targets[least_invasive].max(axis=0)
        else:
            blocking = np.zeros(len(thresh), dtype=bool)
        

        # Apply the indicators
        for i, b, o, s in zip(range(dims), blocking, optimal, sliders):
            # In priority order
            col = 'white'  # default
            col = 'gray' if o else col  # can't be improved
            col = 'orange' if lastbounced[0]==i else col
            col = 'red' if b else col  # mark red if blocking
            s.style=widgets.SliderStyle(handle_color=col)


        # end of callback - release lock
        lock[0] = False

    # Bind the sliders to the observe function
    for s in sliders:
        s.observe(callback, names='value')

    # Display the sliders
    return widgets.VBox(sliders)

make_interactive()  # worst values are on the left

## how to interpret

* Better is always on the left (note sliding Y+ left makes its value larger)

Colours
* white - unconstrained (normal)
* grey - can't improve without a trade off
* orange - failed to achieve requested value
* red - trade-off you could make to improve the orange slider
